## 1. 다국어 RAG 시스템 구축 실습

- 실제 RAG 시스템은 다양한 언어의 문서를 다루거나, 여러 언어로 질문을 받는 상황에 놓일 수 있음.
- 다국어 환경에서 RAG 시스템을 구축하는 방법에 대한 예시 확인

**다국어 RAG의 주요 과제:**
- **언어 불일치**: 질문과 문서의 언어가 다를 경우 검색 성능이 저하될 수 있음.
- **임베딩 모델 선택**: 사용하는 임베딩 모델이 대상 언어들을 얼마나 잘 지원하는지가 중요하며, 교차 언어(cross-lingual) 성능이 좋은 모델이 필요할 수 있음.
- **번역 품질 및 비용**: 자동 번역기를 사용할 경우 번역 품질, 지연 시간, 비용 등을 고려해야 함.

### 1-1 언어 교차(cross-lingual) 검색

- 교차 언어 임베딩 모델은 서로 다른 언어의 텍스트라도 의미가 유사하면 벡터 공간에서 가깝게 위치하도록 학습됨. 
- 이러한 모델을 사용하면 한국어로 질문해도 영어 문서를 검색하거나, 그 반대의 경우도 가능하게 됨.

**전략:**
1.  한국어 문서와 영어 문서를 모두 준비.
2.  교차 언어 성능이 우수한 임베딩 모델 (예: OpenAI `text-embedding-3-small`, HuggingFace `BAAI/bge-m3`, Ollama `bge-m3`)을 선택.
3.  모든 문서를 선택한 임베딩 모델 하나로 임베딩해서 하나의 벡터 저장소에 벡터 저장소에 저장.
4.  사용자 질문(한국어 또는 영어)을 동일한 임베딩 모델로 임베딩하여 벡터 저장소에서 유사 문서를 검색.

**장점:**
- **구현 심플**: 임베딩 모델 하나, 벡터 저장소 하나! 관리 포인트가 적어서 상대적으로 만들기 쉬움.
- **유지보수 용이**: 시스템 구조가 단순해서 유지보수도 편함.


**단점:**
- **임베딩 모델 의존도 UP**: 모델의 교차 언어 성능이 곧 전체 시스템의 성능임. 모델이 별로면 검색 정확도도 뚝... 📉
- **단일 언어 검색보다 약할 수도**: 특정 언어에만 특화된 모델보다는 여러 언어를 다루다 보니, 한국어 질문 -> 한국어 문서 검색 같은 단일 언어 검색 성능이 살짝 떨어질 수 있음.
- **모든 언어 쌍에 완벽하진 않음**: 모델이 특정 언어 쌍(예: 영어-스페인어)에는 강해도, 다른 언어 쌍(예: 한국어-스와힐리어)에는 약할 수 있음. 데이터셋의 한계 때문임.

**꿀팁 & 노하우:** 💡
- **모델 선택 신중하게**: `mTEB (Multilingual Text Embedding Benchmark)` 같은 벤치마크 사이트에서 교차 언어 검색(Cross-Lingual Retrieval) 점수가 높은 모델을 찾아보는 게 좋음. (`BAAI/bge-m3`가 이런 벤치마크에서 좋은 성능을 보여주는 대표적인 모델임!)
- **데이터 중요성**: 학습 데이터에 포함된 언어 및 데이터 양에 따라 성능이 달라지니, 내가 사용할 주요 언어들이 모델 학습 시 잘 다뤄졌는지 확인해보는 것도 좋음.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(1) 다국어 문서 로드 및 전처리` 예시


In [2]:
from glob import glob
import os 
korean_txt_files = glob(os.path.join('../data', '*_KR.txt')) 
english_txt_files = glob(os.path.join('../data', '*_EN.txt'))

print("한국어 텍스트 파일:", korean_txt_files)
print("영어 텍스트 파일:", english_txt_files)

한국어 텍스트 파일: ['../data\\Rivian_KR.txt', '../data\\Tesla_KR.txt']
영어 텍스트 파일: ['../data\\Rivian_EN.txt', '../data\\Tesla_EN.txt']


In [3]:
from langchain_community.document_loaders import TextLoader

def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        # TextLoader는 기본적으로 utf-8을 가정하나, 명시하는 것이 좋음
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

korean_data = load_text_files(korean_txt_files)
english_data = load_text_files(english_txt_files)

print(f"한국어 원본 Document 수: {len(korean_data)}")
print(f"영어 원본 Document 수: {len(english_data)}")

if korean_data:
    print("\n한국어 데이터 샘플 (첫 번째 문서 메타데이터):", korean_data[0].metadata)
    print("한국어 데이터 샘플 (첫 번째 문서 내용 일부):", korean_data[0].page_content[:200])
if english_data:
    print("\n영어 데이터 샘플 (첫 번째 문서 메타데이터):", english_data[0].metadata)
    print("영어 데이터 샘플 (첫 번째 문서 내용 일부):", english_data[0].page_content[:200])

한국어 원본 Document 수: 2
영어 원본 Document 수: 2

한국어 데이터 샘플 (첫 번째 문서 메타데이터): {'source': '../data\\Rivian_KR.txt'}
한국어 데이터 샘플 (첫 번째 문서 내용 일부): 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임. 이들 차량은 "스케이트보드" 플랫

영어 데이터 샘플 (첫 번째 문서 메타데이터): {'source': '../data\\Rivian_EN.txt'}
영어 데이터 샘플 (첫 번째 문서 내용 일부): Rivian Automotive, Inc. is an American electric vehicle manufacturer and automotive technology company.
Founded in 2009 by Robert "RJ" Scaringe, it is headquartered in Irvine, California.
Rivian's mai


In [4]:
from langchain_text_splitters import CharacterTextSplitter

# 문서를 의미 있는 단위(청크)로 쪼개는 작업임.
# CharacterTextSplitter.from_tiktoken_encoder를 쓰면 특정 모델의 토크나이저를 기준으로 글자 수를 계산해서 자름.
# `text-embedding-3-small` 모델은 다국어 처리에 강점이 있어서 이 모델의 토크나이저를 쓰는 건 좋은 선택!
# 왜냐하면, (1) 대규모 다국어 데이터셋으로 학습했고, (2) 다양한 언어의 미묘한 차이까지 고려한 정교한 토크나이저를 사용하기 때문임.
text_splitter_multilingual = CharacterTextSplitter.from_tiktoken_encoder(
    model_name="text-embedding-3-small", # 이 모델의 토크나이저를 사용해서 청크 크기를 계산함
    separator=r"[.!?]\s+",      # 문장 끝맺음(마침표, 느낌표, 물음표) 뒤에 공백이 오는 걸 기준으로 문장을 나눔 (정규식)
    chunk_size=200,              # 목표 청크 크기 (토큰 수). CharacterTextSplitter는 이 값을 정확히 지키진 않을 수 있음.
    chunk_overlap=20,            # 청크끼리 얼마나 겹치게 할 건지 (토큰 수). 문맥 유지를 위해 중요!
    is_separator_regex=True,     # separator가 정규 표현식이라고 알려줌.
    keep_separator=True,        # 구분자(여기선 문장 끝맺음 뒤 공백)는 청크에 포함 안 시킴.
)

korean_docs_split = []
english_docs_split = []

if korean_data:
    korean_docs_split = text_splitter_multilingual.split_documents(korean_data)
if english_data:
    english_docs_split = text_splitter_multilingual.split_documents(english_data)

print(f"한국어 문서를 잘게 쪼갠 청크 수: {len(korean_docs_split)}")
print(f"영어 문서를 잘게 쪼갠 청크 수: {len(english_docs_split)}")

한국어 문서를 잘게 쪼갠 청크 수: 5
영어 문서를 잘게 쪼갠 청크 수: 2


In [5]:
# 한국어 분할 청크 확인 (처음 2개)
if korean_docs_split:
    print("--- 한국어 분할 청크 샘플 ---")
    for i, doc in enumerate(korean_docs_split[:2]):
        print(f"\n[청크 {i+1}] (길이: {len(doc.page_content)}자, 메타데이터: {doc.metadata})")
        print(doc.page_content[:150])
        if len(doc.page_content) > 150: print("[...]")
else:
    print("분할된 한국어 문서가 없습니다.")

--- 한국어 분할 청크 샘플 ---

[청크 1] (길이: 180자, 메타데이터: {'source': '../data\\Rivian_KR.txt'})
리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주
[...]

[청크 2] (길이: 170자, 메타데이터: {'source': '../data\\Rivian_KR.txt'})
. 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 
[...]


In [6]:
# 영어 분할 청크 확인 (처음 2개)
if english_docs_split:
    print("\n--- 영어 분할 청크 샘플 ---")
    for i, doc in enumerate(english_docs_split[:2]):
        print(f"\n[청크 {i+1}] (길이: {len(doc.page_content)}자, 메타데이터: {doc.metadata})")
        print(doc.page_content[:150])
        if len(doc.page_content) > 150: print("[...]")
else:
    print("분할된 영어 문서가 없습니다.")


--- 영어 분할 청크 샘플 ---

[청크 1] (길이: 625자, 메타데이터: {'source': '../data\\Rivian_EN.txt'})
Rivian Automotive, Inc. is an American electric vehicle manufacturer and automotive technology company.
Founded in 2009 by Robert "RJ" Scaringe, it is
[...]

[청크 2] (길이: 741자, 메타데이터: {'source': '../data\\Tesla_EN.txt'})
Tesla, Inc. is an American electric vehicle and clean energy company.
It was co-founded in 2003 by Martin Eberhard and Marc Tarpenning. Elon Musk was 
[...]


**(2) 문서 임베딩 및 벡터 저장소에 저장**

1) 선택한 교차 언어 임베딩 모델들(OpenAI, HuggingFace, Ollama)을 사용하여 한국어와 영어 문서를 함께 임베딩하고, 각 모델별로 ChromaDB 벡터 저장소에 저장
2) 이렇게 하면 각 임베딩 모델의 교차 언어 검색 성능을 비교 가능

- `collection_name`: 벡터 저장소 내에서 특정 문서 그룹을 식별하는 이름이며, 모델별로 다른 이름을 사용 가능.
- `persist_directory`: 벡터 저장소 데이터를 디스크에 저장할 경로이며, 지정하면 저장소가 유지되어 재사용 가능..

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings


# 1 `text-embedding-3-small`은 교차 언어 지원이 꽤 괜찮은 모델임.
embeddings_openai_small_cl = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. Hugging Face 임베딩 모델: `BAAI/bge-m3`는 mTEB 벤치마크에서도 상위권을 차지하는 강력한 다국어 및 교차 언어 모델임.
# `normalize_embeddings: True`는 임베딩 벡터를 정규화해서 유사도 계산 시 코사인 유사도 성능을 높여줌.
embeddings_huggingface_bge_m3_cl = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3", 
    model_kwargs={'device': 'cpu'},# GPU 없으면 'cpu', 있으면 'cuda'로 설정. HuggingFace 모델은 GPU 쓰면 훨씬 빠름!
    encode_kwargs={'normalize_embeddings': True}
)

# Ollama 임베딩 모델 (bge-m3 또는 nomic-embed-text, 교차 언어 지원 가능)
# Ollama 서버 및 해당 모델이 준비되어 있어야 함
embeddings_ollama_bge_m3_cl = None
ollama_ready_cl = False
try:
    embeddings_ollama_bge_m3_cl = OllamaEmbeddings(model="bge-m3") 
    # embeddings_ollama_nomic_cl = OllamaEmbeddings(model="nomic-embed-text")
    print("Ollama 임베딩 모델 (bge-m3) 준비 완료.")
    ollama_ready_cl = True
except Exception as e:
    print(f"Ollama 연결 또는 모델 로드 실패 (교차언어용): {e}")
    print("Ollama (bge-m3) 교차언어 예제를 실행하려면 Ollama 서버를 실행하고 'bge-m3' 모델을 pull 해주세요.")

all_split_docs = korean_docs_split + english_docs_split
print(f"총 분할된 문서(청크) 수: {len(all_split_docs)}")

Ollama 임베딩 모델 (bge-m3) 준비 완료.
총 분할된 문서(청크) 수: 7


In [8]:
# 다국어 벡터 저장소 구축
from langchain_chroma import Chroma

db_openai_cl = None
db_huggingface_cl = None
db_ollama_cl = None

if all_split_docs: # 분할된 문서가 있을 경우에만 실행
    print("OpenAI 임베딩으로 벡터 저장소 구축 중...")
    db_openai_cl = Chroma.from_documents(
        documents=all_split_docs, 
        embedding=embeddings_openai_small_cl,
        collection_name="db_openai_crosslingual_v2", # 컬렉션 이름 변경 또는 기존 삭제 후 생성
        persist_directory="./chroma_db_cl", # 디렉토리 구분
    )
    print(f"OpenAI 벡터 저장소 문서 수: {db_openai_cl._collection.count()}")

    print("\nHugging Face (BAAI/bge-m3) 임베딩으로 벡터 저장소 구축 중...")
    db_huggingface_cl = Chroma.from_documents(
        documents=all_split_docs, 
        embedding=embeddings_huggingface_bge_m3_cl,
        collection_name="db_huggingface_crosslingual_v2",
        persist_directory="./chroma_db_cl",
    )
    print(f"Hugging Face 벡터 저장소 문서 수: {db_huggingface_cl._collection.count()}")

    if ollama_ready_cl and embeddings_ollama_bge_m3_cl:
        print("\nOllama (bge-m3) 임베딩으로 벡터 저장소 구축 중...")
        db_ollama_cl = Chroma.from_documents(
            documents=all_split_docs, 
            embedding=embeddings_ollama_bge_m3_cl,
            collection_name="db_ollama_crosslingual_v2",
            persist_directory="./chroma_db_cl",
        )
        print(f"Ollama 벡터 저장소 문서 수: {db_ollama_cl._collection.count()}")
    else:
        print("\nOllama가 준비되지 않아 Ollama 벡터 저장소 구축을 건너뜁니다.")
else:
    print("분할된 문서가 없어 벡터 저장소 구축을 건너뜁니다.")

OpenAI 임베딩으로 벡터 저장소 구축 중...
OpenAI 벡터 저장소 문서 수: 14

Hugging Face (BAAI/bge-m3) 임베딩으로 벡터 저장소 구축 중...
Hugging Face 벡터 저장소 문서 수: 14

Ollama (bge-m3) 임베딩으로 벡터 저장소 구축 중...
Ollama 벡터 저장소 문서 수: 14


`(3) RAG 성능 비교 `  

- 간단한 RAG 체인을 구성하여 각 임베딩 모델 기반의 벡터 저장소가 한국어 질문과 영어 질문에 대해 어떻게 응답하는지 비교

**RAG 체인 구성 요소:**
- **Retriever**: 벡터 저장소에서 유사 문서를 검색하며, `as_retriever()`로 변환하고, `search_kwargs={'k': 3}`로 상위 2개 문서를 가져오도록 설정.
- **Prompt Template**: LLM에 전달할 프롬프트를 정의하며, 컨텍스트(검색된 문서)와 질문을 포함.
- **LLM**: 질문과 컨텍스트를 바탕으로 최종 답변을 생성할 언어 모델 (예: `ChatOpenAI`)
- **Output Parser**: LLM의 출력(주로 `AIMessage` 객체)에서 실제 텍스트 답변만 추출(`StrOutputParser`)
- **RunnablePassthrough / format_docs**: LangChain Expression Language (LCEL)에서 데이터 흐름을 관리하고, 검색된 `Document` 객체 리스트를 LLM 프롬프트에 적합한 문자열 형태로 변환.

In [9]:
# RAG 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based ONLY on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt_template_cl = ChatPromptTemplate.from_template(template)

In [10]:
# 문서 포맷터 함수: Document 객체 리스트를 단일 문자열로 결합
def format_docs_cl(docs):
    return "\n\n".join([d.page_content for d in docs])

# LLM 모델 생성 (답변 생성용)
llm_cl = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 생성 함수 (벡터 저장소를 인자로 받음)
def create_rag_chain_cl(vectorstore):
    if not vectorstore:
        # 벡터 저장소가 None이면, 실행 불가능한 더미 체인을 반환하거나 예외 처리
        # 여기서는 간단히 None을 반환하여 호출하는 쪽에서 확인하도록 함
        return None
        
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3}) # 상위 3개 문서 검색

    rag_chain = (
        {"context": retriever | format_docs_cl , "question": RunnablePassthrough()} # 검색 및 포맷팅
        | prompt_template_cl  # 프롬프트 적용
        | llm_cl              # LLM으로 답변 생성
        | StrOutputParser()   # 출력 파싱 (텍스트만 추출)
    )
    return rag_chain

In [11]:
# 각 벡터 저장소에 대한 RAG 체인 생성
rag_chain_openai_cl = create_rag_chain_cl(db_openai_cl)
rag_chain_huggingface_cl = create_rag_chain_cl(db_huggingface_cl)
rag_chain_ollama_cl = create_rag_chain_cl(db_ollama_cl)

if rag_chain_openai_cl: print("OpenAI RAG 체인 생성 완료")
else: print("OpenAI RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_huggingface_cl: print("HuggingFace RAG 체인 생성 완료")
else: print("HuggingFace RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_ollama_cl: print("Ollama RAG 체인 생성 완료")
else: print("Ollama RAG 체인 생성 실패 (벡터 저장소 없음 또는 Ollama 준비 안됨)")

OpenAI RAG 체인 생성 완료
HuggingFace RAG 체인 생성 완료
Ollama RAG 체인 생성 완료


In [24]:
# 한국어 쿼리에 대한 성능 평가
query_ko_cl = "테슬라 창업자는 누구인가요?" # 예시 문서에 관련 내용이 있어야 함
print(f"\n--- 한국어 쿼리: '{query_ko_cl}' ---")

if rag_chain_openai_cl:
    output_openai_ko = rag_chain_openai_cl.invoke(query_ko_cl)
    print(f"OpenAI 응답 (KO): {output_openai_ko}")
else:
    print("OpenAI RAG 체인이 없어 실행 불가")

if rag_chain_huggingface_cl:
    output_huggingface_ko = rag_chain_huggingface_cl.invoke(query_ko_cl)
    print(f"Hugging Face (bge-m3) 응답 (KO): {output_huggingface_ko}")
else:
    print("HuggingFace RAG 체인이 없어 실행 불가")

if rag_chain_ollama_cl:
    output_ollama_ko = rag_chain_ollama_cl.invoke(query_ko_cl)
    print(f"Ollama (bge-m3) 응답 (KO): {output_ollama_ko}")
else:
    print("Ollama RAG 체인이 없어 실행 불가")


--- 한국어 쿼리: '테슬라 창업자는 누구인가요?' ---
OpenAI 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.
Hugging Face (bge-m3) 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.
Ollama (bge-m3) 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.


In [13]:
# 영어 쿼리에 대한 성능 평가
query_en_cl = "Who is the founder of Tesla?" # 예시 문서에 관련 내용이 있어야 함
print(f"\n--- 영어 쿼리: '{query_en_cl}' ---")

if rag_chain_openai_cl:
    output_openai_en = rag_chain_openai_cl.invoke(query_en_cl)
    print(f"OpenAI 응답 (EN): {output_openai_en}")
else:
    print("OpenAI RAG 체인이 없어 실행 불가")

if rag_chain_huggingface_cl:
    output_huggingface_en = rag_chain_huggingface_cl.invoke(query_en_cl)
    print(f"Hugging Face (bge-m3) 응답 (EN): {output_huggingface_en}")
else:
    print("HuggingFace RAG 체인이 없어 실행 불가")

if rag_chain_ollama_cl:
    output_ollama_en = rag_chain_ollama_cl.invoke(query_en_cl)
    print(f"Ollama (bge-m3) 응답 (EN): {output_ollama_en}")
else:
    print("Ollama RAG 체인이 없어 실행 불가")


--- 영어 쿼리: 'Who is the founder of Tesla?' ---
OpenAI 응답 (EN): 테슬라는 마틴 에버하드와 마크 타페닝에 의해 공동 창립되었습니다.
Hugging Face (bge-m3) 응답 (EN): 테슬라는 2003년에 마틴 에버하드와 마크 타페닝에 의해 공동 창립되었습니다.
Ollama (bge-m3) 응답 (EN): 테슬라는 마틴 에버하드와 마크 타페닝에 의해 공동 창립되었습니다.


### 1-2 언어 감지 및 자동번역 통합 

- 이 전략은 주로 단일 언어(예: 한국어)로 된 문서 저장소를 가지고 있을 때, 외국어로 들어오는 질문을 처리하기 위해 사용됨.
반대로, 영어 문서 저장소에 한국어 질문을 받고 싶을 때도 쓸 수 있음!
핵심은 **질문과 답변을 필요에 따라 번역**하는 거임

**전략:**
1.  **언어 감지**: 사용자 질문의 언어를 감지 (예: `langdetect` 라이브러리 사용).
2.  **질문 번역**: 감지된 질문 언어가 문서 저장소의 주 언어와 다르면, 질문을 문서 저장소의 언어로 번역(예: `deepl` API 사용).
3.  **RAG 처리**: 번역된 질문을 사용하여 일반적인 RAG 체인을 통해 답변을 생성 (이때 답변은 문서 저장소의 언어로 생성됨).
4.  **답변 번역**: 생성된 답변의 언어가 원래 질문의 언어와 다르면, 답변을 원래 질문의 언어로 다시 번역

**장점:**
- 단일 언어에 최적화된 임베딩 모델과 LLM을 활용하여 해당 언어에서의 검색 및 답변 생성 품질을 극대화할 수 있음.
- 다양한 언어의 사용자 질문을 지원할 수 있음.

**단점:**
- **번역 품질 의존성**: 번역기의 성능에 따라 전체 시스템의 품질이 크게 좌우됨. 번역 오류는 잘못된 검색 결과나 부정확한 답변으로 이어질 수 있음.
- **지연 시간 증가**: 질문과 답변 번역 과정에서 추가적인 API 호출로 인해 전체 응답 시간이 늘어남.
- **번역 비용**: 상용 번역 API(예: DeepL, Google Translate) 사용 시 비용이 발생.
- **언어 감지 오류**: 언어 감지가 정확하지 않으면 불필요하거나 잘못된 번역이 발생 가능성.



**꿀팁 & 노하우:** 💡
- **번역기 선택**: DeepL은 번역 품질이 좋다고 알려져 있지만 유료임. Google Translate API, Papago API 등 다른 옵션도 있고, 심지어 자체 번역 모델(오픈소스 모델 fine-tuning 등)을 구축할 수도 있음 (전문가 영역!).
- **언어 감지 라이브러리**: `langdetect`는 사용하기 쉽지만 가끔 짧은 텍스트에서 틀릴 수 있음. `fastText`의 언어 식별 모델은 더 정확하지만, 모델 파일이 크고 설정이 좀 더 필요함.
- **번역 캐싱**: 똑같은 문장을 여러 번 번역하지 않도록, 이미 번역한 결과는 저장해뒀다가 재사용(캐싱)하면 비용과 시간을 아낄 수 있음.
- **타겟 언어 명확화**: "문서 저장소의 주 언어"를 명확히 정하고, 모든 질문을 그 언어로 통일시키는 것이 관리하기 편함.

`(1) 한국어 문서 벡터저장소 로드 (또는 생성)`  

- 한국어 문서만으로 구성된 벡터 저장소가 이미 있다고 가정.
- 만약 없다면, 한국어 문서(`korean_docs_split`)와 한국어에 강한 임베딩 모델(예: `OpenAIEmbeddings`, 또는 한국어 특화 HuggingFace 모델)을 사용하여 새로 생성할 수 있음.

In [14]:
# 한국어 문서로 저장되어 있는 벡터 저장소 로드 또는 생성
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_for_ko_store = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# 기존에 'chroma_test' 컬렉션이 한국어 데이터로 만들어져 있다고 가정.
# 없다면, korean_docs_split을 사용해 새로 생성해야 함.
# 예시
db_korean_only = Chroma.from_documents(documents=korean_docs_split, embedding=embeddings_for_ko_store, 
                                       collection_name="korean_store_v1", persist_directory="./chroma_db_ko_only")

In [15]:
COLLECTION_NAME_KO_ONLY = "db_openai_crosslingual_v2" # (한국어+영어 데이터 포함)
PERSIST_DIR_KO_ONLY = "./chroma_db_cl"

try:
    # 여기서는 1-1에서 만든 db_openai_cl (한국어+영어 문서 포함)을 재사용
    # 순수 한국어 저장소를 원한다면, 해당 저장소를 로드하거나 새로 만들어야 함.
    vectorstore_ko_trans = Chroma(
        embedding_function=embeddings_for_ko_store,
        collection_name=COLLECTION_NAME_KO_ONLY, # 1-1에서 사용한 OpenAI 컬렉션 이름
        persist_directory=PERSIST_DIR_KO_ONLY    # 1-1에서 사용한 디렉토리
    )
    print(f"'{COLLECTION_NAME_KO_ONLY}' 벡터 저장소 로드 완료. 문서 수: {vectorstore_ko_trans._collection.count()}")
    # 이 저장소는 실제로는 한국어와 영어가 섞여있지만, 지금은 한국어 중심 저장소로 간주하고 진행
    # 이상적으로는 순수 한국어 문서로 구성된 저장소를 사용하는 것이 이 시나리오에 더 적합
except Exception as e:
    print(f"벡터 저장소 '{COLLECTION_NAME_KO_ONLY}' 로드 실패: {e}")
    print("이전 단계에서 해당 이름의 컬렉션이 생성되었는지, 경로가 올바른지 확인하세요.")
    vectorstore_ko_trans = None

'db_openai_crosslingual_v2' 벡터 저장소 로드 완료. 문서 수: 14


**언어 감지 및 번역 도구 설정**

- `langdetect`: 텍스트의 언어를 감지
- `deepl`: 고품질 번역을 제공하는 API 서비스

In [16]:
import os
import deepl
from langdetect import detect, DetectorFactory

# 언어 감지 결과의 일관성을 위해 시드 설정 (선택 사항)
DetectorFactory.seed = 0

deepl_api_key = os.getenv('DEEPL_API_KEY')
translator = None
if deepl_api_key:
    translator = deepl.Translator(deepl_api_key)
    print("DeepL 번역기 초기화 완료.")
else:
    print("DEEPL_API_KEY가 설정되지 않았습니다. 번역 기능을 사용하려면 API 키를 설정하세요.")

DeepL 번역기 초기화 완료.


In [17]:
def detect_and_translate(text: str, target_lang_deepl: str = 'KO', target_lang_detect: str = 'ko'):
    if not translator:
        print("번역기가 설정되지 않아 원본 텍스트를 반환합니다.")
        # 번역기 없어도 언어 감지는 시도
        try:
            detected_lang = detect(text)
            return text, detected_lang
        except Exception as e:
            print(f"언어 감지 실패: {e}. 원본 텍스트 및 'unknown' 반환.")
            return text, "unknown"

    try:
        detected_lang = detect(text) # langdetect는 'ko', 'en' 등 소문자 코드를 반환
    except Exception as e:
        print(f"언어 감지 실패: {e}. 원본 텍스트 사용.")
        return text, "unknown"

    # target_lang_deepl ('EN', 'KO', 'EN-US' 등)을 langdetect 스타일 ('en', 'ko')로 변환하여 비교하기 위함
    simple_target_deepl_code = ''
    temp_target_deepl_upper = target_lang_deepl.upper()
    if temp_target_deepl_upper.startswith('EN'): # 'EN', 'EN-US', 'EN-GB' 등
        simple_target_deepl_code = 'en'
    elif temp_target_deepl_upper == 'KO':
        simple_target_deepl_code = 'ko'
    # 다른 언어에 대한 규칙 추가 가능
    else:
        # 일반적인 경우 (예: 'FR', 'JA') DeepL 코드는 2자리이므로 앞 두 글자를 소문자로 사용
        if len(target_lang_deepl) >= 2:
            simple_target_deepl_code = target_lang_deepl[:2].lower()
        else: # 매우 드문 경우
            simple_target_deepl_code = target_lang_deepl.lower()

    # 감지된 언어(소문자)와 실제 목표 언어(단순화된 소문자 코드)를 비교
    if detected_lang.lower() != simple_target_deepl_code:
        print(f"번역 필요: 감지된 언어 '{detected_lang}' -> 목표 언어 '{target_lang_deepl}'")
        try:
            # DeepL은 'EN-US', 'EN-GB' 등을 지원. 단순히 'EN'으로 하면 기본 'EN-US'로 될 수 있음.
            # langdetect가 'en'을 반환하면, DeepL의 'EN-US'로 매핑하는 것이 안전할 수 있음.
            target_lang_deepl_actual = target_lang_deepl # 기본값 설정
            if target_lang_deepl.upper() == 'EN': # 만약 영어로 번역해야 한다면
                target_lang_deepl_actual = 'EN-US' # 미국 영어로 지정 (DeepL 기본 동작과 일치시키거나 명시적 지정)
            # 다른 target_lang_deepl 값들은 그대로 사용 (예: 'KO', 'FR', 'DE')
            # (위의 if문에서 target_lang_deepl_actual = target_lang_deepl 로 이미 설정됨)

            result = translator.translate_text(text, target_lang=target_lang_deepl_actual)
            return str(result), detected_lang
        except Exception as e:
            print(f"번역 실패: {e}. 원본 텍스트 사용.")
            return text, detected_lang
    else: # 감지된 언어와 (단순화된) 목표 언어가 같으면 번역 안 함
        print(f"번역 불필요: 감지된 언어 '{detected_lang}' (단순화: '{detected_lang.lower()}') == 목표 언어 '{target_lang_deepl}' (단순화: '{simple_target_deepl_code}'). 원본 텍스트 반환.")
        return text, detected_lang

In [18]:
# 문서 번역 테스트
if translator:
    sample_text_en = "Hello. How are you today?"
    translated_text, detected_lang_sample = detect_and_translate(sample_text_en, target_lang_deepl='KO') # target_lang_detect 기본값 'ko' 사용
    print(f"원본 (EN): {sample_text_en}")
    print(f"감지된 언어: {detected_lang_sample}")
    print(f"번역된 텍스트 (KO): {translated_text}")
    print("----")

    sample_text_ko = "안녕하세요. 오늘 기분이 어떠신가요?"
    translated_text_en, detected_lang_sample_ko = detect_and_translate(sample_text_ko, target_lang_deepl='EN') # target_lang_detect 기본값 'ko' 사용
    print(f"원본 (KO): {sample_text_ko}")
    print(f"감지된 언어: {detected_lang_sample_ko}")
    print(f"번역된 텍스트 (EN-US): {translated_text_en}")
else:
    print("DeepL 번역기가 없어 번역 테스트를 건너뜁니다.")

번역 필요: 감지된 언어 'en' -> 목표 언어 'KO'
원본 (EN): Hello. How are you today?
감지된 언어: en
번역된 텍스트 (KO): 안녕하세요. 오늘은 어떠세요?
----
번역 필요: 감지된 언어 'ko' -> 목표 언어 'EN'
원본 (KO): 안녕하세요. 오늘 기분이 어떠신가요?
감지된 언어: ko
번역된 텍스트 (EN-US): hello. How are you feeling today?


**(2) 번역 통합 RAG 체인 성능 평가**

이제 `detect_and_translate` 함수를 RAG 체인 앞뒤에 붙여서, 번역 통합 RAG 시스템을 만들어볼 거임.
이 체인은 다음과 같이 작동함:
1. 외국어 질문이 들어오면 -> 한국어(우리 문서 저장소 언어)로 번역
2. 번역된 한국어 질문으로 -> 한국어 컨텍스트 기반 답변 생성 (답변도 한국어)
3. 생성된 한국어 답변을 -> 원래 질문 언어로 다시 번역해서 사용자에게 전달!

In [19]:
lang_rag_chain_translate = None
if vectorstore_ko_trans: # 한국어(중심) 벡터 저장소가 제대로 로드됐는지 먼저 확인
    retriever_ko_trans = vectorstore_ko_trans.as_retriever(search_kwargs={'k': 2}) # 여기서도 문서 2개 가져옴

    # RAG 체인 자체는 1-1에서 만든 것과 거의 동일함 (프롬프트, LLM, 포맷터 재사용)
    # 달라지는 건 이 체인을 어떻게 호출하고 결과를 처리하는지임!
    lang_rag_chain_translate = (
        {"context": retriever_ko_trans | format_docs_cl , "question": RunnablePassthrough()}
        | prompt_template_cl # 1-1에서 정의한 프롬프트 (컨텍스트 기반 답변, 모르면 모른다고 하기)
        | llm_cl             # 1-1에서 정의한 LLM (gpt-4o-mini)
        | StrOutputParser()
    )
    print("번역 기능을 통합할 RAG 체인 (한국어 기반) 준비 완료!")
else:
    print("한국어(중심) 벡터 저장소가 없어서 번역 통합 RAG 체인을 만들 수가 없어요.")

번역 기능을 통합할 RAG 체인 (한국어 기반) 준비 완료!


In [20]:
# 번역을 포함해서 전체 RAG 과정을 실행하는 함수
def run_lang_rag_chain_with_translation(query: str, rag_chain_to_use, 
                                        document_language_deepl='KO', 
                                        document_language_detect='ko'):
    if not rag_chain_to_use:
        return "RAG 체인이 준비되지 않았습니다. 먼저 체인을 만들어주세요."

    print(f"\n[처리 시작] 원본 질문: '{query}'")

    # 1. 질문 언어 감지 및 문서 저장소 언어(여기선 한국어)로 번역
    #    DeepL 번역기가 없으면, 번역 없이 원본 질문과 감지된 언어 코드만 받음.
    translated_query, original_lang_code_lc = detect_and_translate(query, 
                                                                   target_lang_deepl=document_language_deepl, 
                                                                   target_lang_detect=document_language_detect)
    print(f"   질문 언어 감지 결과: '{original_lang_code_lc}'")
    if original_lang_code_lc.lower() != document_language_detect.lower() and query != translated_query:
        print(f"   문서 저장소 언어({document_language_deepl})로 번역된 질문: {translated_query}")
    
    # 2. 번역된 (또는 원본) 질문으로 RAG 체인 실행 -> 답변은 문서 저장소 언어로 생성됨
    print(f"   RAG 시스템에 '{translated_query}' 전달하여 답변 생성 중...")
    output_in_doc_lang = rag_chain_to_use.invoke(translated_query)
    print(f"   RAG 시스템 답변 (문서 저장소 언어 - {document_language_deepl}): {output_in_doc_lang}")
    
    # 3. 생성된 답변을 원래 질문 언어로 다시 번역 (필요한 경우, DeepL 번역기 있을 때만)
    if original_lang_code_lc.lower() != document_language_detect.lower() and query != translated_query: # 원본 질문이 번역되었었다면
        # DeepL이 지원하는 target_lang 코드로 변환 필요.
        # langdetect는 'en', 'ja' 등을 반환하지만, DeepL은 'EN-US', 'JA' 등을 기대함.
        target_lang_deepl_for_answer = original_lang_code_lc.upper() # 일단 대문자로
        if target_lang_deepl_for_answer == 'EN': # langdetect가 'en'이면 보통 'EN-US'나 'EN-GB'를 써야 함
            target_lang_deepl_for_answer = 'EN-US' # 예시로 미국 영어 사용
        # 다른 언어(예: 'JA', 'FR')들은 보통 2자리 코드가 그대로 쓰임. 필요시 규칙 추가.

        print(f"   답변을 원본 질문 언어('{target_lang_deepl_for_answer}')로 다시 번역 시도...")
        final_output, _ = detect_and_translate(output_in_doc_lang, 
                                               target_lang_deepl=target_lang_deepl_for_answer, 
                                               target_lang_detect=target_lang_deepl_for_answer.lower()[:2]) # 답변 번역 시엔 target_lang_detect를 DeepL 코드 기준으로 맞춰줌
        if output_in_doc_lang != final_output:
             print(f"   최종 번역된 답변 ({target_lang_deepl_for_answer}): {final_output}")
        else:
             print(f"   답변 번역 시도했으나, (아마도 번역기 부재로) 원본 답변과 동일합니다.")
        return final_output
    else: # 원래 질문이 문서 저장소 언어였으면 번역 없이 바로 반환
        print(f"   원본 질문이 이미 문서 저장소 언어와 같으므로, 답변 번역 없이 반환합니다.")
        return output_in_doc_lang

In [21]:
if lang_rag_chain_translate: # 체인이 준비되었을 때만 테스트!
    # 시나리오 1: 한국어 질문 (번역 거의 불필요)
    query_ko_trans = "테슬라 창업자는 누구인가요?"
    # print(f"\n--- [번역 통합 RAG] 한국어 질문 테스트: '{query_ko_trans}' ---")
    output_ko_final = run_lang_rag_chain_with_translation(query_ko_trans, lang_rag_chain_translate, 
                                                          document_language_deepl='KO', document_language_detect='ko')
    print(f"최종 답변 (KO): {output_ko_final}")

    # 시나리오 2: 영어 질문 (질문: EN->KO 번역, 답변: KO->EN-US 번역 필요)
    query_en_trans = "Who is the founder of Tesla?"
    # print(f"\n--- [번역 통합 RAG] 영어 질문 테스트: '{query_en_trans}' ---")
    output_en_final = run_lang_rag_chain_with_translation(query_en_trans, lang_rag_chain_translate, 
                                                          document_language_deepl='KO', document_language_detect='ko')
    print(f"최종 답변 (EN-US): {output_en_final}")

    # 시나리오 3: 일본어 질문 (DeepL 무료 버전은 지원 언어 제한적일 수 있음. API 키 플랜 확인!)
    # (질문: JA->KO 번역, 답변: KO->JA 번역 필요)
    query_ja_trans = "テスラの創業者は誰ですか？"
    # print(f"\n--- [번역 통합 RAG] 일본어 질문 테스트: '{query_ja_trans}' ---")
    output_ja_final = run_lang_rag_chain_with_translation(query_ja_trans, lang_rag_chain_translate, 
                                                          document_language_deepl='KO', document_language_detect='ko')
    print(f"최종 답변 (JA): {output_ja_final}")
else:
    print("\n번역 통합 RAG 체인이 준비되지 않아서 실행을 건너뜁니다. 😢")
    if not translator:
        print("   특히 DeepL 번역기가 준비되지 않은 것 같네요. API 키를 확인해주세요!")


[처리 시작] 원본 질문: '테슬라 창업자는 누구인가요?'
번역 불필요: 감지된 언어 'ko' (단순화: 'ko') == 목표 언어 'KO' (단순화: 'ko'). 원본 텍스트 반환.
   질문 언어 감지 결과: 'ko'
   RAG 시스템에 '테슬라 창업자는 누구인가요?' 전달하여 답변 생성 중...
   RAG 시스템 답변 (문서 저장소 언어 - KO): 마틴 에버하드와 마크 타페닝입니다.
   원본 질문이 이미 문서 저장소 언어와 같으므로, 답변 번역 없이 반환합니다.
최종 답변 (KO): 마틴 에버하드와 마크 타페닝입니다.

[처리 시작] 원본 질문: 'Who is the founder of Tesla?'
번역 필요: 감지된 언어 'en' -> 목표 언어 'KO'
   질문 언어 감지 결과: 'en'
   문서 저장소 언어(KO)로 번역된 질문: 테슬라의 창립자는 누구인가요?
   RAG 시스템에 '테슬라의 창립자는 누구인가요?' 전달하여 답변 생성 중...
   RAG 시스템 답변 (문서 저장소 언어 - KO): 마틴 에버하드와 마크 타페닝입니다.
   답변을 원본 질문 언어('EN-US')로 다시 번역 시도...
번역 필요: 감지된 언어 'ko' -> 목표 언어 'EN-US'
   최종 번역된 답변 (EN-US): Martin Everhard and Mark Tappening.
최종 답변 (EN-US): Martin Everhard and Mark Tappening.

[처리 시작] 원본 질문: 'テスラの創業者は誰ですか？'
번역 필요: 감지된 언어 'ja' -> 목표 언어 'KO'
   질문 언어 감지 결과: 'ja'
   문서 저장소 언어(KO)로 번역된 질문: 테슬라의 창업자는 누구인가요?
   RAG 시스템에 '테슬라의 창업자는 누구인가요?' 전달하여 답변 생성 중...
   RAG 시스템 답변 (문서 저장소 언어 - KO): 마틴 에버하드와 마크 타페닝입니다.
   답변을 원본 질문 언어('JA')로 다시 번역 시도...
번역 필요: 감지된

### 1-3 언어 감지 및 벡터저장소 라우팅

- 이 전략은 각 언어별로 최적화된 RAG 시스템(문서 저장소, 임베딩 모델 등)을 따로 구축하고, 사용자 질문의 언어를 감지해서 알맞은 시스템으로 "길안내(라우팅)" 해주는 방식임.


**전략:**
1.  **언어별 벡터 저장소 구축**: 한국어 문서는 한국어 특화 임베딩 모델로, 영어 문서는 영어 특화 (또는 좋은 다국어) 임베딩 모델로 각각 별도의 벡터 저장소를 만듦. (필요하면 언어별로 LLM도 다르게 설정 가능!)
2.  **언어 감지**: 사용자 질문의 언어를 감지
3.  **라우팅**: 감지된 언어에 해당하는 벡터 저장소 및 관련 RAG 체인으로 질문을 전달
4.  **RAG 처리**: 선택된 RAG 체인에서 답변을 생성

**장점:**
- 각 언어에 최적화된 임베딩과 RAG 파이프라인을 사용할 수 있어, 해당 언어 내에서의 검색 및 답변 품질이 높을 수 있음.
- 번역 과정이 없어 지연 시간과 번역 비용이 발생하지 않음/ (단, 교차 언어 질의는 직접 지원하지 않음).

**단점:**
- **여러 벡터 저장소 관리**: 지원하는 언어 수만큼 벡터 저장소와 RAG 체인을 관리해야 하므로 복잡성이 증가.
- **교차 언어 검색 미지원**: 기본적으로는 질문 언어와 동일한 언어의 문서만 검색(예: 영어 질문으로는 영어 문서만 검색). 교차 언어 검색을 지원하려면 각 저장소에서 사용하는 임베딩 모델 자체가 교차 언어 성능이 뛰어나거나, 추가적인 번역 로직이 필요.
- **언어 감지 정확도 중요**: 언어 감지가 잘못되면 엉뚱한 저장소로 라우팅되어 성능이 저하


**꿀팁 & 노하우:** 💡
- **라우팅 로직 설계**: 단순히 언어 코드(`ko`, `en`)로 분기하는 것 외에도, 특정 키워드나 질문 패턴에 따라 다른 RAG 체인으로 보낼 수도 있음. (LangChain의 `RouterChain` 같은 걸 활용 가능)
- **Fallback 전략**: 언어 감지가 애매하거나 지원하지 않는 언어의 질문이 들어왔을 때, 어떻게 처리할지 미리 정해두는 게 좋음. (예: 기본 언어(영어) RAG로 보내거나, "지원하지 않는 언어입니다" 안내)
- **점진적 확장**: 처음부터 모든 언어를 다 지원하려고 하기보다, 주요 타겟 언어 2~3개부터 시작해서 점차 늘려나가는 방식이 현실적임.

**(1) 언어별 벡터 저장소 생성**

이제 진짜 언어별로 독립된 벡터 저장소를 만들어 볼 거임.
- 한국어 문서(`korean_docs_split`)는 한국어 처리에 유리한 임베딩 모델로!
- 영어 문서(`english_docs_split`)는 영어 처리에 유리하거나 좋은 다국어 임베딩 모델로!
각각 별도의 ChromaDB 컬렉션에 저장할 거임.

**임베딩 모델 선택 예시:**
- **한국어 문서용**: 1-1에서 사용했던 HuggingFace의 `BAAI/bge-m3`를 그대로 써보겠음. (이 모델은 다국어지만 한국어 성능도 좋음. 만약 더 한국어에 특화된 모델이 있다면 그걸 써도 좋음! 예: `ko-sroberta-multitask`)
- **영어 문서용**: 1-1에서 사용했던 Ollama의 `bge-m3`를 써보겠음. (만약 Ollama가 안된다면 OpenAI 모델 등으로 대체 가능)

**중요**: 각 언어에 "최적"인 임베딩을 찾는 것은 실험이 필요할 수 있음! 여기서는 사용 가능한 모델로 구성함.

In [22]:
# 한국어 문서와 영어 문서를 각각 다른 벡터 저장소(컬렉션)에 저장할 거임.
db_korean_route = None
db_english_route = None

# === 한국어 문서용 벡터 저장소 ===
# 여기서는 1-1에서 정의한 HuggingFace bge-m3 임베딩(embeddings_huggingface_bge_m3_cl)을 재사용.
if korean_docs_split and embeddings_huggingface_bge_m3_cl:
    print("한국어 문서용 벡터 저장소(HuggingFace bge-m3 사용) 생성 시작...")
    db_korean_route = Chroma.from_documents(
        documents=korean_docs_split, 
        embedding=embeddings_huggingface_bge_m3_cl, 
        collection_name="db_korean_for_routing_v3", # 라우팅용 한국어 컬렉션 이름
        persist_directory="./chroma_db_routed", # 라우팅 전용 저장 폴더
    )
    print(f"   한국어 라우팅용 벡터 저장소에 저장된 문서(청크) 수: {db_korean_route._collection.count()}")
else:
    print("분할된 한국어 문서가 없거나 HuggingFace 임베딩 모델이 준비 안돼서 한국어 라우팅용 벡터 저장소는 스킵함.")

# === 영어 문서용 벡터 저장소 ===
# 여기서는 1-1에서 정의한 Ollama bge-m3 임베딩(embeddings_ollama_bge_m3_cl)을 재사용.
# Ollama가 안되면 OpenAI 임베딩 (embeddings_openai_small_cl) 등으로 대체 가능.
embedding_for_english_route = None
if ollama_ready_cl and embeddings_ollama_bge_m3_cl:
    embedding_for_english_route = embeddings_ollama_bge_m3_cl
    print("\n영어 문서용 임베딩으로 Ollama bge-m3를 사용합니다.")
elif embeddings_openai_small_cl: # Ollama 없으면 OpenAI 모델로 대체 시도
    embedding_for_english_route = embeddings_openai_small_cl
    print("\nOllama bge-m3를 사용할 수 없어, 영어 문서용 임베딩으로 OpenAI text-embedding-3-small을 대신 사용합니다.")
else:
    print("\n영어 문서용 임베딩 모델을 찾을 수 없습니다.")

if english_docs_split and embedding_for_english_route:
    print("영어 문서용 벡터 저장소 생성 시작...")
    db_english_route = Chroma.from_documents(
        documents=english_docs_split, 
        embedding=embedding_for_english_route, 
        collection_name="db_english_for_routing_v3", # 라우팅용 영어 컬렉션 이름
        persist_directory="./chroma_db_routed", # 라우팅 전용 저장 폴더 (위와 동일 폴더, 다른 컬렉션)
    )
    print(f"   영어 라우팅용 벡터 저장소에 저장된 문서(청크) 수: {db_english_route._collection.count()}")
elif not english_docs_split:
    print("\n분할된 영어 문서가 없어서 영어 라우팅용 벡터 저장소는 스킵함.")
else:
    print("\n영어 문서용 임베딩 모델이 준비 안돼서 영어 라우팅용 벡터 저장소는 스킵함.")

한국어 문서용 벡터 저장소(HuggingFace bge-m3 사용) 생성 시작...
   한국어 라우팅용 벡터 저장소에 저장된 문서(청크) 수: 10

영어 문서용 임베딩으로 Ollama bge-m3를 사용합니다.
영어 문서용 벡터 저장소 생성 시작...
   영어 라우팅용 벡터 저장소에 저장된 문서(청크) 수: 4


**(2) 라우팅 RAG 체인 성능 평가**

- 이제 언어 감지 결과를 보고 "이 질문은 한국어 RAG 팀으로!", "저 질문은 영어 RAG 팀으로!" 하고 똑똑하게 나눠주는 함수를 만들어서 테스트해는 것임.


In [23]:
# 각 언어별 RAG 체인 생성 (create_rag_chain_cl 함수 재사용)
rag_chain_korean_routed = create_rag_chain_cl(db_korean_route)
rag_chain_english_routed = create_rag_chain_cl(db_english_route)

if rag_chain_korean_routed: print("라우팅용 한국어 RAG 체인 생성 완료")
else: print("라우팅용 한국어 RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_english_routed: print("라우팅용 영어 RAG 체인 생성 완료")
else: print("라우팅용 영어 RAG 체인 생성 실패 (벡터 저장소 없음)")

def run_route_rag_chain(query: str):
    try:
        detected_query_lang = detect(query) # 질문 언어 감지
        print(f"감지된 질문 언어: {detected_query_lang}")
    except Exception as e:
        print(f"질문 언어 감지 실패: {e}. 기본 체인(한국어) 시도 또는 에러 반환.")
        # detected_query_lang = 'unknown' # 또는 기본 언어 설정
        return f"질문 언어 감지 실패: {e}"
    
    if detected_query_lang == 'ko' and rag_chain_korean_routed:
        print("한국어 RAG 체인으로 라우팅합니다.")
        return rag_chain_korean_routed.invoke(query)
    elif detected_query_lang == 'en' and rag_chain_english_routed:
        print("영어 RAG 체인으로 라우팅합니다.")
        return rag_chain_english_routed.invoke(query)
    else:
        # 지원하지 않는 언어 또는 해당 언어 체인이 없는 경우
        if detected_query_lang == 'ko' and not rag_chain_korean_routed:
            return "한국어 RAG 체인이 준비되지 않았습니다."
        if detected_query_lang == 'en' and not rag_chain_english_routed:
            return "영어 RAG 체인이 준비되지 않았습니다."
        return f"지원하지 않는 언어({detected_query_lang})이거나 해당 언어의 RAG 체인이 없습니다. 한국어 또는 영어로 질문해주세요."
    
# 한국어 쿼리에 대한 성능 평가
query_ko_route = "테슬라 창업자는 누구인가요?"
print(f"\n--- 실행: 라우팅 한국어 질문 '{query_ko_route}' ---")
output_ko_route = run_route_rag_chain(query_ko_route)
print(f"결과 (KO 라우팅): {output_ko_route}")

# 영어 쿼리에 대한 성능 평가
query_en_route = "Who is the founder of Tesla?"
print(f"\n--- 실행: 라우팅 영어 질문 '{query_en_route}' ---")
output_en_route = run_route_rag_chain(query_en_route)
print(f"결과 (EN 라우팅): {output_en_route}")

# 지원하지 않는 언어 테스트 (예: 일본어)
query_ja_route = "テスラの創業者は誰ですか？"
print(f"\n--- 실행: 라우팅 일본어 질문 '{query_ja_route}' ---")
output_ja_route = run_route_rag_chain(query_ja_route)
print(f"결과 (JA 라우팅): {output_ja_route}")

라우팅용 한국어 RAG 체인 생성 완료
라우팅용 영어 RAG 체인 생성 완료

--- 실행: 라우팅 한국어 질문 '테슬라 창업자는 누구인가요?' ---
감지된 질문 언어: ko
한국어 RAG 체인으로 라우팅합니다.
결과 (KO 라우팅): 마틴 에버하드와 마크 타페닝입니다.

--- 실행: 라우팅 영어 질문 'Who is the founder of Tesla?' ---
감지된 질문 언어: en
영어 RAG 체인으로 라우팅합니다.
결과 (EN 라우팅): Tesla was co-founded in 2003 by Martin Eberhard and Marc Tarpenning.

--- 실행: 라우팅 일본어 질문 'テスラの創業者は誰ですか？' ---
감지된 질문 언어: ja
결과 (JA 라우팅): 지원하지 않는 언어(ja)이거나 해당 언어의 RAG 체인이 없습니다. 한국어 또는 영어로 질문해주세요.
